[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA2/blob/main/lab6/lab6-parte1.ipynb)
# Práctica 6: Redes neuronales convolucionales - Parte 1 - FF vs CNN


### Pre-requisitos. Instalar paquetes

Para la primera parte de este Laboratorio 6 necesitaremos, además de torch, el módulo torchvision para cargar los datos. Además, como habitualmente, fijaremos la semilla aleatoria para asegurar la reproducibilidad de los experimentos.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import transforms
import os
import numpy as np
import random
from matplotlib import pyplot as plt

# Fijamos la semilla para reproducibilidad
seed = 1234567
os.environ['PYTHONHASHSEED'] = str(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


### Carga del conjunto de datos

En esta ocasión trabajaremos con el conjunto de imágenes *mnist*, que representa dígitos escritos a mano. Debemos indicar a nuestro dataloader que haga lotes de 128 elementos.

In [ ]:
# Cargamos MNIST usando torchvision
transform = transforms.Compose([
    transforms.ToTensor(), # convierte a tensor y escala a [0,1]
])

train_val = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# TODO - Divide train_val en train y val (80/20)

# TODO - Crea los DataLoaders
batch_size = 128
train_loader = ...
val_loader = ...
test_loader = ...

NUM_CLASSES = 10
IMAGE_SHAPE = (1, 28, 28)

# Para comprobar que se ha cargado tomamos un elemento y lo mostramos
im_batch, label_batch = next(iter(train_loader))
ej_imagen = im_batch[0]
plt.imshow(ej_imagen.squeeze())
plt.xlabel(f'Label: {label_batch[0].item()}')
plt.show()


## Ajustando los datos con un red neuronal feed-forward

Vamos a modelar los datos con una red feed-forward que tenga capas de 40, 25 y 16 unidades (todas con activación ReLU). Debemos tener en cuenta que las imágenes son tensores de dimensión 3 (su `shape` es (1,28,28)), mientras que la entrada de nuestras capas Dense debe ser un tensor de dimensión 1. Para adecuar la entrada a lo que necesitamos, vamos a "aplanar" los tensores de las imágenes, que pasarán de `shape` (28,28,1) a `shape` (784). Para ello aplanaremos el lote de (128,1,28,28) a (128, 784) en la función `forward`.

Por último, la salida de nuestro modelo debe tener tantas componentes como clases distintas tiene el conjunto. Como queremos que la salida aproxime la probabilidad de las distintas clases, lo habitual sería poner una función de activación *softmax*, pero en este caso, por razones de eficiencia del entrenamiento, es mejor dejar una salida lineal y posteriormente usar la función de pérdida `nn.CrossEntropyLoss` que espera las salidas vienen en ese formato.

In [ ]:
class MLP(nn.Module):
    # TODO - Completa la clase siguiendo las instrucciones dadas en la celda anterior.

### Entrenamiento del modelo
Vamos a establecer la función de pérdida, el optimizador (Adam con el LR por defecto) y la métrica que nos servirá para evaluar el rendimiento del modelo entrenado (precisión categórica).

Como intentamos predecir una clase entre varias, nuestra función de pérdida debe ser la [entropía cruzada](https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html). Le indicaremos que la salida de nuestra red como *logits* y las etiquetas con el valor numérico de la clase (no en one-hot encoding).

In [ ]:
# Evaluación sobre el conjunto de test
def evaluate(model, dataloader, loss_fn):
    # TODO - Completa la función para que devuelva la pérdida y la tasa de acierto


def train_model(model, train_loader, loss_fn, optimizer, num_epochs=16):
    # TODO - Completa la función para que haga el entrenamiento devuelva el histórico de pérdidas y tasas de acierto

# TODO - Define el modelo, función de pérdida y optimizador (Adam con lr=0.001) y haz el entrenamiento durante 16 epochs
ff_model = ...
loss_fn = ...
optimizer = ...
train_losses, val_losses = ...

### Verificación del rendimiento
Aprovecharemos el conjunto de test para comprobar la capacidad de generalización de nuestro modelo. Puedes también visualizar la curva de entrenamiento para identificar potenciales problemas con el entrenamiento.

In [ ]:
# TODO - Evalúa sobre el conjunto de test

Si todo ha ido correctamente deberías haber obtenido un valor de precisión sobre el conjunto de test comparable a los obtenidos con los conjuntos de entrenamiento y validación, lo que indica que el modelo generaliza bien a otros datos del conjunto original pero... ¿tenemos un buen modelo?

Vamos a comprobar la robustez del modelo haciendo pequeños desplazamientos de las imágenes originales. Utilizaremos un pequeño modelo que aplique una traslación aleatoria de hasta un 10% del tamaño de la imagen a cada una de las imágenes de test. Nos ayudaremos del módulo `transforms` incluido en `torchvision` y en particular de [`transforms.RandomAffine`](https://docs.pytorch.org/vision/main/generated/torchvision.transforms.RandomAffine.html) para hacer traslacciones de hasta un 10% del tamaño de imagen en cualquier dirección.

In [ ]:
# Data augmentation: aplicamos una pequeña traslación usando torchvision.transforms
transform_translated = transforms.Compose([
    # TODO - Utiliza RandomAffine para hacer traslacciones de hasta un 10% en cualquier dirección
    transforms.ToTensor(),
])

# Creamos un dataset de test transformado
test_translated = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_translated)
test_translated_loader = ...

Comprobemos ahora la precisión sobre este nuevo conjunto de imágenes que han sido ligeramente desplazadas.

In [ ]:
# TODO - Evalúa sobre el conjunto de test desplazado

Si todo ha ido bien, deberías haber comprobado que estas pequeñas traslaciones son suficientes para que la precisión del modelo baje sustancialmente. Las redes feed-forward no son robustas ante este tipo de perturbaciones.

## Comparativa con una red convolucional

Declara ahora un modelo convolucional con la siguiente arquitectura:
 1. [Convolución 2D](https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) de 8 filtros y tamaño de kernel 3, con activación ReLU
 1. [Pooling 2D](https://docs.pytorch.org/docs/2.8/generated/torch.nn.MaxPool2d.html) tomando el máximo de cada grupo de 2x2
  1. [Convolución 2D](https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) de 8 filtros y tamaño de kernel 3, con activación ReLU
 1. [Pooling 2D](https://docs.pytorch.org/docs/2.8/generated/torch.nn.MaxPool2d.html) tomando el máximo de cada grupo de 2x2
 1. Capa Densa (requiere aplanado previo) de 32 unidades y activación ReLU
 1. Capa de salida
 
Define el nuevo modelo, entrénalo y haz las verificaciones posteriores para observar la diferencia.

In [ ]:
class ConvModel(nn.Module):
    # TODO - Completa la clase según las instrucciones

# TODO - Define el modelo, función de pérdida y optimizador (Adam con lr=0.001) y haz el entrenamiento durante 16 epochs
conv_model = ...
loss_fn = ...
optimizer = ...
train_losses, val_losses = ...

# TODO - Haz las evaluaciones como en hiciste con ff_model

### Reflexiones sobre la comparativa
 - ¿Qué has observado en el rendimiento?
 - ¿Cuántos parámetros tiene la red convolucional respecto a la *feed-forward*
 - ¿Cómo ha cambiado el tiempo de ejecución?
 - ¿Es más robusta frente a los desplazamientos esta red?